In [20]:
from google.colab import files
files.upload()


Saving dataset.csv to dataset (1).csv
Saving symptom_Description.csv to symptom_Description (1).csv
Saving symptom_precaution.csv to symptom_precaution (1).csv
Saving Symptom-severity.csv to Symptom-severity (1).csv


{'dataset (1).csv': b'Disease,Symptom_1,Symptom_2,Symptom_3,Symptom_4,Symptom_5,Symptom_6,Symptom_7,Symptom_8,Symptom_9,Symptom_10,Symptom_11,Symptom_12,Symptom_13,Symptom_14,Symptom_15,Symptom_16,Symptom_17\nFungal infection,itching, skin_rash, nodal_skin_eruptions, dischromic _patches,,,,,,,,,,,,,\nFungal infection, skin_rash, nodal_skin_eruptions, dischromic _patches,,,,,,,,,,,,,,\nFungal infection,itching, nodal_skin_eruptions, dischromic _patches,,,,,,,,,,,,,,\nFungal infection,itching, skin_rash, dischromic _patches,,,,,,,,,,,,,,\nFungal infection,itching, skin_rash, nodal_skin_eruptions,,,,,,,,,,,,,,\nFungal infection, skin_rash, nodal_skin_eruptions, dischromic _patches,,,,,,,,,,,,,,\nFungal infection,itching, nodal_skin_eruptions, dischromic _patches,,,,,,,,,,,,,,\nFungal infection,itching, skin_rash, dischromic _patches,,,,,,,,,,,,,,\nFungal infection,itching, skin_rash, nodal_skin_eruptions,,,,,,,,,,,,,,\nFungal infection,itching, skin_rash, nodal_skin_eruptions, dischromic 

In [21]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score


In [22]:
df = pd.read_csv("dataset.csv")
df.fillna("None", inplace=True)

print("Dataset loaded successfully")
print("Shape:", df.shape)


Dataset loaded successfully
Shape: (4920, 18)


In [23]:
symptom_columns = df.columns[1:]  # exclude Disease column

all_symptoms = set()
for col in symptom_columns:
    all_symptoms.update(df[col].unique())

all_symptoms.discard("None")
all_symptoms = sorted(list(all_symptoms))

print("Total symptoms available:", len(all_symptoms))


Total symptoms available: 131


In [24]:
X = df.drop("Disease", axis=1)
y = df["Disease"]

# One-hot encoding for symptoms (TEXT → NUMERIC)
X_encoded = pd.get_dummies(X)

# Label encoding for disease
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)


In [25]:
X_train, X_test, y_train, y_test = train_test_split(
    X_encoded, y_encoded, test_size=0.2, random_state=42
)


In [26]:
model = RandomForestClassifier(
    n_estimators=200,
    random_state=42
)

model.fit(X_train, y_train)


RandomForestClassifier(n_estimators=200, random_state=42)

In [27]:
y_pred = model.predict(X_test)
print("Model Accuracy:", accuracy_score(y_test, y_pred))


Model Accuracy: 1.0


In [28]:
print("\n📋 SELECT SYMPTOMS FROM THE LIST BELOW:\n")
for i, symptom in enumerate(all_symptoms, start=1):
    print(f"{i}. {symptom}")



📋 SELECT SYMPTOMS FROM THE LIST BELOW:

1.  abdominal_pain
2.  abnormal_menstruation
3.  acidity
4.  acute_liver_failure
5.  altered_sensorium
6.  anxiety
7.  back_pain
8.  belly_pain
9.  blackheads
10.  bladder_discomfort
11.  blister
12.  blood_in_sputum
13.  bloody_stool
14.  blurred_and_distorted_vision
15.  breathlessness
16.  brittle_nails
17.  bruising
18.  burning_micturition
19.  chest_pain
20.  chills
21.  cold_hands_and_feets
22.  coma
23.  congestion
24.  constipation
25.  continuous_feel_of_urine
26.  continuous_sneezing
27.  cough
28.  cramps
29.  dark_urine
30.  dehydration
31.  depression
32.  diarrhoea
33.  dischromic _patches
34.  distention_of_abdomen
35.  dizziness
36.  drying_and_tingling_lips
37.  enlarged_thyroid
38.  excessive_hunger
39.  extra_marital_contacts
40.  family_history
41.  fast_heart_rate
42.  fatigue
43.  fluid_overload
44.  foul_smell_of urine
45.  headache
46.  high_fever
47.  hip_joint_pain
48.  history_of_alcohol_consumption
49.  increased_app

In [29]:
selection = input(
    "\nEnter symptom numbers separated by comma (example: 2,7,15): "
)

selected_numbers = [int(i.strip()) for i in selection.split(",")]
user_symptoms = [all_symptoms[i-1] for i in selected_numbers]

print("\n✅ You selected these symptoms:")
print(user_symptoms)



Enter symptom numbers separated by comma (example: 2,7,15): 2,7,15

✅ You selected these symptoms:
[' abnormal_menstruation', ' back_pain', ' breathlessness']


In [30]:
input_vector = np.zeros(len(X_encoded.columns))

for symptom in user_symptoms:
    matching_columns = [
        col for col in X_encoded.columns if col.endswith("_" + symptom)
    ]
    for col in matching_columns:
        input_vector[X_encoded.columns.get_loc(col)] = 1

input_df = pd.DataFrame([input_vector], columns=X_encoded.columns)


In [31]:
prediction = model.predict(input_df)
predicted_disease = label_encoder.inverse_transform(prediction)[0]

print("\n🩺 Predicted Disease:", predicted_disease)



🩺 Predicted Disease: Common Cold


In [32]:
desc_df = pd.read_csv("symptom_Description.csv")
prec_df = pd.read_csv("symptom_precaution.csv")

description = desc_df[
    desc_df["Disease"] == predicted_disease
]["Description"].values[0]

precautions = prec_df[
    prec_df["Disease"] == predicted_disease
].values[0][1:]

print("\n📘 Disease Description:")
print(description)

print("\n🛡️ Precautions:")
for i, p in enumerate(precautions, start=1):
    print(f"{i}. {p}")



📘 Disease Description:
The common cold is a viral infection of your nose and throat (upper respiratory tract). It's usually harmless, although it might not feel that way. Many types of viruses can cause a common cold.

🛡️ Precautions:
1. drink vitamin c rich drinks
2. take vapour
3. avoid cold food
4. keep fever in check
